In [1]:
from library.functions_to_abstract_data import extract_qm9_data
from torch_geometric.datasets import QM9

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the QM9 dataset or install if not already installed
dataset = QM9(root='../data/QM9')
print("Number of molecules:", len(dataset))

Extracting ../data/QM9/raw/qm9.zip
Processing...
100%|██████████| 133885/133885 [02:54<00:00, 769.44it/s] 
Done!


Number of molecules: 130831


In [3]:
# Convert to panda dataframe
df = extract_qm9_data(dataset)
df

,index,smiles,num_atoms,mu,alpha,HOMO,LUMO,gap,r2,zpve,...,U298_atom,H298_atom,G298_atom,A,B,C,atoms,pos,edge_idx,edge_attr
0,0,[H]C([H])([H])[H],5,0.0000,13.210000,-10.549854,3.186453,13.736308,35.364101,1.217682,...,-17.286823,-17.389656,-16.151918,157.711807,157.709976,157.706985,"[tensor(6), tensor(1), tensor(1), tensor(1), t...","[[tensor(-0.0127), tensor(1.0858), tensor(0.00...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
1,1,[H]N([H])[H],4,1.6256,9.460000,-6.993326,2.255824,9.249149,26.156300,0.934929,...,-12.082129,-12.159273,-11.246005,293.609741,293.541107,191.393967,"[tensor(7), tensor(1), tensor(1), tensor(1)]","[[tensor(-0.0404), tensor(1.0241), tensor(0.06...","[[tensor(0), tensor(0), tensor(0), tensor(1), ...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
2,2,[H]O[H],3,1.8511,6.310000,-7.967494,1.869422,9.836916,19.000200,0.581643,...,-9.278811,-9.330214,-8.733849,799.588135,437.903870,282.945465,"[tensor(8), tensor(1), tensor(1)]","[[tensor(-0.0344), tensor(0.9775), tensor(0.00...","[[tensor(0), tensor(0), tensor(1), tensor(2)],...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
3,3,[H]C#C[H],4,0.0000,16.280001,-7.741639,1.376896,9.118535,59.524799,0.730381,...,-16.792231,-16.869347,-15.862634,0.000000,35.610035,35.610035,"[tensor(6), tensor(6), tensor(1), tensor(1)]","[[tensor(0.5995), tensor(0.), tensor(1.)], [te...","[[tensor(0), tensor(0), tensor(1), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0..."
4,4,[H]C#N,3,2.8937,12.990000,-9.806983,0.519737,10.329442,48.747601,0.451736,...,-13.135290,-13.186666,-12.520096,0.000000,44.593884,44.593884,"[tensor(6), tensor(7), tensor(1)]","[[tensor(-0.0133), tensor(1.1325), tensor(0.00...","[[tensor(0), tensor(0), tensor(1), tensor(2)],...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130826,130826,[H]C1([H])[C@@]2([H])[N@@H+]3[C@@]2([H])[C@@]2...,16,1.6637,69.370003,-6.133446,1.600029,7.733476,760.747192,3.466894,...,-70.028702,-70.414177,-64.734833,3.594830,2.198990,1.904230,"[tensor(6), tensor(6), tensor(6), tensor(6), t...","[[tensor(1.2387), tensor(-0.5096), tensor(1.18...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
130827,130827,[H]C1([H])[N@@H+]2[C@@]3([H])[C@]2([H])[C@]2([...,16,1.2976,69.519997,-6.511684,1.654452,8.168858,762.635376,3.469316,...,-69.915634,-70.301140,-64.621742,3.656480,2.142370,1.904390,"[tensor(6), tensor(7), tensor(6), tensor(6), t...","[[tensor(0.7296), tensor(-0.5677), tensor(-1.4...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
130828,130828,[H]C1([H])[N@@H+]2[C@@]3([H])[C@]2([H])[C@]2([...,17,1.2480,73.599998,-6.076303,1.959220,8.035522,780.355286,3.822057,...,-72.801025,-73.212242,-67.177277,3.671180,2.143140,1.895010,"[tensor(6), tensor(7), tensor(6), tensor(6), t...","[[tensor(-1.1819), tensor(1.4150), tensor(0.87...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."
130829,130829,[H]C1([H])[N@H+]2[C@@]3([H])[C@@]14C([H])([H])...,18,1.9576,77.400002,-5.774256,2.397323,8.171579,803.190430,4.142171,...,-78.368141,-78.805077,-72.433289,3.528450,2.151310,1.865820,"[tensor(6), tensor(7), tensor(6), tensor(6), t...","[[tensor(-1.4356), tensor(1.3633), tensor(0.31...","[[tensor(0), tensor(0), tensor(0), tensor(0), ...","[[tensor(1.), tensor(0.), tensor(0.), tensor(0..."


In [4]:
# List of all attributes of a molecule present in the QM9 database
df.columns

Index(['index', 'smiles', 'num_atoms', 'mu', 'alpha', 'HOMO', 'LUMO', 'gap',
       'r2', 'zpve', 'U0', 'U298', 'H298', 'G298', 'Cv', 'U0_atom',
       'U298_atom', 'H298_atom', 'G298_atom', 'A', 'B', 'C', 'atoms', 'pos',
       'edge_idx', 'edge_attr'],
      dtype='object')

In [5]:
# Check if any NaN values are present
print(df.isna().sum())

index        0
smiles       0
num_atoms    0
mu           0
alpha        0
HOMO         0
LUMO         0
gap          0
r2           0
zpve         0
U0           0
U298         0
H298         0
G298         0
Cv           0
U0_atom      0
U298_atom    0
H298_atom    0
G298_atom    0
A            0
B            0
C            0
atoms        0
pos          0
edge_idx     0
edge_attr    0
dtype: int64


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# --- Step 1: Choose features and target ---
features = ['num_atoms','mu','alpha','r2','zpve','U0','U298','H298','G298','Cv']
X = df[features].values
y = df['gap'].values

# --- Step 2: Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Step 3: Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Step 4: Train Linear Regression ---
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# --- Step 5: Predict and evaluate ---
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Linear Regression MAE (eV): {mae:.3f}")
print(f"Linear Regression R²: {r2:.3f}")

# --- Step 6: Inspect coefficients ---
print("\nFeature coefficients:")
for feat, coef in zip(features, model.coef_):
    print(f"{feat}: {coef:.3f}")


Linear Regression MAE (eV): 0.706
Linear Regression R²: 0.539

Feature coefficients:
num_atoms: 2.287
mu: -0.067
alpha: -1.232
r2: 0.490
zpve: -1.260
U0: 60004.976
U298: -30543.389
H298: -24689.690
G298: -4771.157
Cv: 1.561
